In [13]:
#第八章 数据规整：聚合、合并和重塑
#8.1 层次化索引（hierarchical indexing)
import pandas as pd
import numpy as np
data = pd.Series(np.random.randn(9),index = [['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])
#赛有MultiIndex索引的Series的格式
data

a  1    2.068370
   2   -1.166438
   3    1.791956
b  1   -0.785375
   3   -0.234119
c  1   -0.037843
   2   -0.505151
d  2   -0.156174
   3   -0.608210
dtype: float64

In [15]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [5]:
data['b']

1   -0.027939
3    0.877019
1   -0.003008
dtype: float64

In [7]:
data['b':'c']

b  1   -0.027939
   3    0.877019
   1   -0.003008
c  2   -0.919906
   2   -0.323929
   3    1.224480
dtype: float64

In [16]:
data.loc[['b','d']]

b  1   -0.785375
   3   -0.234119
d  2   -0.156174
   3   -0.608210
dtype: float64

In [17]:
#在“内层”中进行选取：
data.loc[:,2]

a   -1.166438
c   -0.505151
d   -0.156174
dtype: float64

In [19]:
#层次化索引在数据重塑和基于分组的操作（如透视表生成）中有重要的作用
#如通过unstack方法将数据重新安排到一个DataFrame中
data.unstack()

,1,2,3
a,2.068370,-1.166438,1.791956
b,-0.785375,NaN,-0.234119
c,-0.037843,-0.505151,NaN
d,NaN,-0.156174,-0.608210


In [20]:
#unstack的逆运算是stack
data.unstack().stack()

a  1    2.068370
   2   -1.166438
   3    1.791956
b  1   -0.785375
   3   -0.234119
c  1   -0.037843
   2   -0.505151
d  2   -0.156174
   3   -0.608210
dtype: float64

In [21]:
#对于一个DataFrame，每条轴都可以由分层索引：
frame = pd.DataFrame(np.arange(12).reshape((4,3)),index = [['a','a','b','b'],[1,2,1,2]],columns = [['Ohio','Ohio','Colorado'],['Green','Red','Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [23]:
#名称都可以由名字
frame.index.names = ['key1','key2']
frame.columns.names = ['state','color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [24]:
#有了部分列索引，就可以轻松选取列分组：
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [31]:
#单独创建MultiIndex然后复用,上面那个DataFrame中的（带有分级名称）列可以这样创建：
import numpy as np
import pandas as pd
#参考书中没有pd，实战中需要加入pd
pd.MultiIndex.from_arrays([['Ohio','Ohio','Colorado'],['Green','Red','Green']],names = ['state','color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           codes=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

In [33]:
#重排与分级排序
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [29]:
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

NameError: name 'MultiIndex' is not defined

In [35]:
#sort_index根据单个级别中的值对数据进行排序
frame.sort_index(level = 1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [36]:
frame.swaplevel(0,1).sort_index(level = 0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [37]:
#根据级别汇总统计
#根据DataFrame和Series的描述和汇总统计都有一个level选项，可用于指定在某条轴上求和的级别
frame.sum(level = 'key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [39]:
frame.sum(level = 'color',axis = 1)
#这其实用了DataFrame的列进行索引

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [40]:
#使用DataFrame的列进行索引
frame = pd.DataFrame({'a':range(7),'b':range(7,0,-1),'c':['one','one','one','two','two','two','two'],'d':[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [42]:
#DataFrame的set_index函数会将其一个或多个列转换为行索引，并创建一个新的DataFrame
frame2 = frame.set_index(['c','d'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [43]:
#默认情况下，那些列会从DataFrame中移除，但也可以将其保留下来：
frame.set_index(['c','d'],drop = False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [45]:
#reset_index的功能跟set_index刚好相反，层次化索引的级别会被转移到列里面：
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [ ]:
#合并数据集
#pandas对象中的数据可以通过以下方式进行合并：
#pandas.merge可根据一个或多个键将不同DataFrame中的行连接起来，类似于SQL的join操作
#pandas.concat可以沿着一条轴将多个对象堆叠到一起
#combine_first可以将重复数据拼接在一起，用一个对象中的值填充另一个对象中的缺失值

In [47]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'],'data2':range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [48]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [49]:
#这是多对一的合并：
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [50]:
#最好明确指定使用哪个列进行重叠
pd.merge(df1,df2,on = 'key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [52]:
#如果两个对象的列名不同，也可以分别进行指定：
df3 = pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],'data1':range(7)})
df4 = pd.DataFrame({'rkey':['a','b','d'],'data2':range(3)})
pd.merge(df3,df4,left_on = 'lkey',right_on = 'rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [53]:
#默认情况下pd.merge取得是“内连接”，结果中的键是交集；其他方式还有“left"、”right"、“outer”，外连接求取的是键的并集，组合了左连接和右连接的效果：
pd.merge(df1,df2, how = "outer")

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [55]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [56]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [77]:
#df2添加一行
df2.loc[3] = ['d',3]
df2.loc[4] = ['d',4]
df2

,key,data2
0,a,0
1,b,1
2,d,2
3,d,3
4,d,4


In [78]:
#inplace=True表示直接对原表进行修改
df1_1 = df1.drop(6,axis = 0)
df1_1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5


In [79]:
pd.merge(df1_1,df2,on = 'key',how = 'left')

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0


In [80]:
pd.merge(df1_1,df2,how = 'inner')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0


In [82]:
left = pd.DataFrame({'key1':['foo','foo','bar'],
                    'key2': ['one', 'two', 'one'],
                    'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
#根据多个键进行合并，传入一个由列名组成的列表即可
pd.merge(left,right,on = ['key1','key2'],how = 'outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [83]:
#merge有一个更实用的suffixes选项，用于指定附加到左右两个DataFrame对象的重叠列名上的字符串：
pd.merge(left,right,on = 'key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [84]:
pd.merge(left,right,on = 'key1',suffixes = ('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [91]:
#索引上的合并
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a','b'])
pd.merge(left1,right1,left_on = 'key',right_index = True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [93]:
#得到并集
pd.merge(left1,right1,left_on = 'key',right_index = True,how= 'outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [98]:
#层次化索引的数据合并默认是多建合并：
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001,
                               2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio',
                              'Ohio', 'Ohio','Ohio'],
                             [2001, 2000, 2000, 2000, 2001,2002]],
                      columns=['event1', 'event2'])
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [99]:
#以列表的形式指明用作合并键的多个列（注意用how='outer'对重复索引值的处理）：
pd.merge(lefth,righth,left_on = ['key1','key2'],right_index = True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [100]:
pd.merge(lefth,righth,left_on = ['key1','key2'],right_index = True,how = 'outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [102]:
#同时使用合并双方的索引
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13,
14]],index=['b', 'c', 'd', 'e'],columns=['Missouri', 'Alabama'])
pd.merge(left2,right2,how = 'outer',left_index = True,right_index = True)                   

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [103]:
#DataFrame还有一个便捷的join实例方法，可以方便地实现按索引合并，但要求没有重叠地列：
left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [104]:
#历史遗留原因，DataFrame默认是使用左连接，保留左边表地行索引。
left1.join(right1,on = 'key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [106]:
#对于简单地索引合并，可向join传入一组DataFrame
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.],[16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York','Oregon'])
left2.join([right2,another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [107]:
left2.join([right2,another],how = 'outer')

E:\Anaconda\Setup\lib\site-packages\pandas\core\frame.py:6848: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


In [108]:
#轴向连接
#另一种的数据合并运算也被称为连接（concatenation），绑定（binding）或堆叠（stacking），
#Numpy的concatenation函数可以用Numpy数组来做：
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [110]:
np.concatenate([arr,arr],axis = 1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [109]:
#对于pandas对象（如Series和DataFrame），带有标签的轴能够进一步推广数组的连接运算


array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [111]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
#使用concat将值和索引粘合在一起：
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [112]:
#默认情况下，concat是在axis=0上工作的，最终生成一个新的Series，如果传入axis = 1,则会生成一个DataFrame
pd.concat([s1,s2,s3],axis = 1)

E:\Anaconda\Setup\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [116]:
#这种情况下，另外的轴上没有重叠，从索引的有序并集（外连接）上就可看出，传入join='inner'即可得到他们的交集：
s4 = pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [118]:
pd.concat([s1,s3],axis = 1)

E:\Anaconda\Setup\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,NaN
b,1.0,NaN
f,NaN,5.0
g,NaN,6.0


In [119]:
pd.concat([s1, s4], axis=1, join='inner')
#f,g消失了是由于使用了'join='inner'选项

,0,1
a,0,0
b,1,1


In [120]:
#可通过join_axes指定要在其他轴上使用的索引：
pd.concat([s1,s4],axis = 1,join_axes = [['a','c','b','e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [121]:
#不过有个问题，参与连接的片段在结果中区分不开，假设想要在连接轴上创建一个
#层次化索引，使用keys参数即可达到这个目的：
result= pd.concat([s1,s1,s3],keys=['one','two','three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [122]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [124]:
#如果沿着axis=1对Series进行合并，则key就会成为DataFrame的列头：
pd.concat([s1, s2, s3], axis=1, keys=['one','two', 'three'])

E:\Anaconda\Setup\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [125]:
#同样的情况也可适用于DataFrame对象：
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a','b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

E:\Anaconda\Setup\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [127]:
#如果传入的不是列表而是一个字典，则字典的键就会被当作keys选项的值：
pd.concat({'level1':df1,'level2':df2},axis = 1)

E:\Anaconda\Setup\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [128]:
#有两个管理层次化索引创建方式的参数，如：names参数命名创建的轴级别
pd.concat([df1,df2],axis = 1,keys=['level1','level2'],names=['upper','lower'])

E:\Anaconda\Setup\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [130]:
#关于DataFrame的问题是，DataFrame的行索引不包括任何相关数据：
df1 = pd.DataFrame(np.random.randn(3,4),columns = ['a','b','c','d'])
df2 = pd.DataFrame(np.random.randn(2,3),columns = ['b','d','a'])
#在这种情况下，传入ignore_index = True即可
pd.concat([df1,df2],ignore_index = True)

E:\Anaconda\Setup\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


,a,b,c,d
0,0.724229,1.444350,0.802436,-1.007680
1,-1.924704,-2.021777,0.509740,-0.646315
2,0.088101,-0.151790,1.360001,0.356506
3,1.593157,-0.046768,NaN,0.760414
4,-0.866680,0.832824,NaN,-0.799062


In [131]:
#合并重叠数据
#索引全部或部分重叠的两个数据集
a = pd.Series([np.nan,2.5, np.nan, 3.5, 4.5, np.nan],index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a),dtype = np.float64),index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1]=np.nan
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [132]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [133]:
#Series有一个combine_first方法，实现：
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [134]:
#DataFrame数据的combine_first具有同样的功能，可以看作是：
#用传递对象中的数据为调用对象的缺失数据"打补丁"
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [135]:
#重塑（reshape)和轴向旋转(pivot)
#重塑层次化索引的主要功能有二：
#1.stack：将数据的列“旋转”为行
#2.unstack：将数据的行“旋转”为列
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio','Colorado'],name='state'),
                    columns=pd.Index(['one', 'two','three'],name='number'))
data                             

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [137]:
#使用stack方法将列转换为行，得到一个Series：
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [138]:
#对于一个层次化索引的Series，可以使用unstack将其重排为一个DataFrame
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [139]:
#默认情况下，unstack操作的是最内层（stack也是如此），传入分层级别的编号或名称即可对其它级别进行unstack操作：
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [140]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [142]:
#如果不是所有的级别值都能在各分组中找到的话，则unstack操作可能会引入缺失数据：
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1,s2],keys = ['one','two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [143]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [145]:
#stack默认会滤除缺失数据，因此该运算是可逆的
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [146]:
#如果设置dropna=False参数则不可逆
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [147]:
#在对DataFrame进行unstack操作时，作为旋转轴的级别将会成为结果中的最低级别：
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'],name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [148]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [149]:
#当调用stack，可指定轴的名字：
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [150]:
#将“长格式”旋转为“宽格式”
#多数时间序列数据通常时以所谓的“长格式”（long）或“堆叠格式”（stacked）存储在数据库和CSV中的：
data = pd.read_csv('examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [152]:
periods = pd.PeriodIndex(year=data.year,quarter=data.quarter,name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'],name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0:'value'})
pivoted = ldata.pivot('date', 'item', 'value')
pivoited

AttributeError: 'DataFrame' object has no attribute 'year'

In [153]:
#将“宽格式”旋转为“长格式”
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],'A': [1, 2, 3],'B': [4, 5, 6],'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [154]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [155]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [156]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [157]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [158]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [159]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


In [ ]:
#8.4:总结：
#现在已经掌握了pandas数据导入、清洗、重塑（日常还需回顾学习）